In [23]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pandas as pd
import wlcave as wlcave #need to path correctly!

## Import Files From Simulation:

In [2]:
file = "C:\\users/Thomas/wlcsim_membrane/data/2.10.21_NoInteractions_2hrs_r11v0.txt"
data = np.loadtxt(file) #D:\\myfiles\welcome.txt


## Our Approach to Calculating Percolation Paths Through a Polymer Membrane:

### Defining The System:

In our system we have random copolymers with solvent molecules. Polymers are represented by beads of types 1 and 2 and solvent molecules are individual beads of type 0. No bead can overlap another. 

### Defining Percolation:

A percolation is a pathway through the polymer network that solvent molecules can move through. To form a percolation a solvent molecule will need to have at least one other solvent molecule adjacent to it within some path radius $r$

### What are we trying to learn from the system:

The goal in developing this code is to determine quantitatively the amount of percolation in a membrane. 

### Models for Percolation:
**1. Fire Spread Model:** In this model we measure the percolation paths by tracing the spots where the solvent molecules continue to be adjacent to the previous... It's easier to explain with a picture

## What we need to calculate:

Starting with the final equilibrium state we have x,y,z postion of a bead and it's identity (and also methylation state). We want to calculate percolation paths which will be represented by root mean square. 

#### What Percolation Model do we want to use:

1. Fire spead model:

Proceedure:
1. Pick a bead type
2. pick a random bead of that bead type
3. Check surround bead types to see if they are the same
    a. Pick a radius to search in


# Note: for some reason if you declare two different PolymerNetworks, they can interfere w/ each other (change values)

In [3]:
def generate_test_data(length=10,width=10,height=10,datapoints=500):
    X = np.random.random(datapoints)*length
    Y = np.random.random(datapoints)*width
    Z = np.random.random(datapoints)*height
    Beads = np.random.randint(3,size=(datapoints)).astype('uint8')
    Beads
    return X,Y,Z,Beads

#generate_test_data()

In [4]:
class Bead:
    '''
    '''
    def __init__(self, x, y, z, bead_type):
        self.x = x
        self.y = y
        self.z = z
        self.bead_type = bead_type
        


In [5]:
class Polymer:
    '''
    '''
    
    def __init__(self, bead_list):
        self.bead_list = bead_list
    
    def print_bead_list(self):
        for i, bead in enumerate(self.bead_list):
            print("Bead " + str(i) + ": " + str(bead.x) + ", " + str(bead.y) + ", " + str(bead.z) + " Type: " + str(bead.bead_type))
   

In [6]:
class PolymerNetwork:
    """
    This represents the postions of bead and their type (time invariant)
    
    Parameters
    ----------
    X : float [array] X 
    """
    
    def __init__(self,X,Y,Z, Beads, beads_per_polymer=40, persistence_length=2.03, bead_list=[], polymer_list=[]):
        self.X = X
        self.Y = Y
        self.Z = Z
        self.Beads = Beads
        self.beads_per_polymer = beads_per_polymer
        self.bead_list = bead_list
        self.polymer_list = polymer_list
        self.persistence_length = persistence_length
        
        self.density = Beads.shape[0]/((max(X)-min(X)) * (max(Y)-min(Y)) * (max(Z)-min(Z)))
        
    def visualize(self):
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.scatter3D(self.X[Beads==0], self.Y[Beads==0], self.Z[Beads==0],color='y')
        ax.scatter3D(self.X[Beads==1], self.Y[Beads==1], self.Z[Beads==1],color='r')
        ax.scatter3D(self.X[Beads==2], self.Y[Beads==2], self.Z[Beads==2],color='b')
        
    def nearest_neighbor(self,bead_num,radius):
        '''
        Find the average number of beads that surround a bead of the same type within the radius specified
        '''
        X_bead = self.X[Beads==bead_num]
        Y_bead = self.Y[Beads==bead_num]
        Z_bead = self.Z[Beads==bead_num]
        
        total_neighbors = 0
        for i in range(Beads[Beads==bead_num].shape[0]):
            point = np.array([X_bead[i],Y_bead[i],Z_bead[i]])
            displacement = np.sqrt((X_bead - point[0])**2 + (Y_bead - point[1])**2 + (Z_bead - point[2])**2)
            num_neighbor_points = displacement[displacement<=radius].shape[0] -1
            
            total_neighbors += num_neighbor_points

        return total_neighbors/(i+1)
    
    def create_beads(self):
        if len(self.bead_list) != 0:
            raise Exception("bead list already populated")
        for i in range(len(self.X)):
            bead = Bead(self.X[i], self.Y[i], self.Z[i], self.Beads[i])
            self.bead_list.append(bead)
    
    def print_bead_list(self):
        for i, bead in enumerate(self.bead_list):
            print("Bead " + str(i) + ": " + str(bead.x) + ", " + str(bead.y) + ", " + str(bead.z) + " Type: " + str(bead.bead_type))
    
    def create_polymers(self):
        if len(self.polymer_list) != 0:
            raise Exception("polymer list already populated")
        if len(self.bead_list) == 0:
            raise Exception("Need to run create_beads before create_polymers")
        for i in range(self.beads_per_polymer, len(self.bead_list) + 1, self.beads_per_polymer):
            polymer = Polymer(self.bead_list[i - self.beads_per_polymer : i])
            self.polymer_list.append(polymer)
        
    def print_polymer_list(self, verbose=False):
        for i, poly in enumerate(self.polymer_list):
            print("POLYMER " + str(i))
            if verbose:
                poly.print_bead_list()

In [7]:
'''
Determine the mean square interbead distance
Input: PolymerNetwork
Return: A DataFrame where the columns are a specific polymer in the network, rows are the number of "jumps" n,
        and the values are the average distance squared bewteen beads n jumps away (averaged over all different jumps 
        of size n possible in the polymer)
mean square distance b/w beads
'''
def r2(polymer_network):
    if len(polymer_network.bead_list) == 0:
        raise Exception("Need to run create_beads")
    if len(polymer_network.polymer_list) == 0:
        raise Exception("Need to run create_polymers")
    num_poly = len(polymer_network.polymer_list)
    avg_n_jumps = pd.DataFrame(index=range(polymer_network.beads_per_polymer), columns=range(num_poly))
    
    for i, poly in enumerate(polymer_network.polymer_list):
        # creating matrix of (inter-bead distances)^2 for a given polymer
        num_beads = len(poly.bead_list)
        bead_dist = np.zeros((num_beads, num_beads))    
        for j, bead1 in enumerate(poly.bead_list):
            for k, bead2 in enumerate(poly.bead_list):
                position1 = np.array([bead1.x, bead1.y, bead1.z])
                position2 = np.array([bead2.x, bead2.y, bead2.z])
                displacement_squared = np.sum((position2 - position1)**2)
                bead_dist[j][k] = displacement_squared
        
        # get diagonals, store in the dataframe
        n_jump_dist_avg = np.zeros(num_beads)
        for d in range(num_beads):
            diag = np.diag(bead_dist, d)
            n_jump_dist_avg[d] = np.average(diag)    # populated with average displacements of d jumps
        
        avg_n_jumps[i] = n_jump_dist_avg
        if (i % 500 == 0):
            print("processed polymer " + str(i))
    
    #average the distance b/w chain ends, normailize
    r2_ave = np.average(avg_n_jumps.iloc[num_beads-1])/(2 * polymer_network.persistence_length)**2
    
    return r2_ave, avg_n_jumps

In [8]:
#TEST DATA:

#X,Y,Z,Beads = generate_test_data()
#PNet_Test = PolymerNetwork(X,Y,Z,Beads, beads_per_polymer = 10)
#PNet_Test.create_beads()
#PNet_Test.create_polymers()
#PNet_Test.density

In [9]:
#PNet_Test.visualize()

In [10]:
#PNet_Test.nearest_neighbor(bead_num=0,radius=0.4)

In [11]:
#radii = np.linspace(0,15,100)
#for r in radii:
#    plt.plot(r,PNet_Test.nearest_neighbor(bead_num=0,radius=r),'y.')
#plt.xlabel('Radius')
#plt.ylabel('Average Number of Neighbor Points')

In [12]:
#x = r2(PNet_Test)
#x  # roughly random dist (no polymer chain)

In [13]:
#REAL DATA:

beads = np.zeros(len(data[:, 0]))   # no chemical identity data
no_interactions = PolymerNetwork(data[:, 0], data[:, 1], data[:, 2], beads, beads_per_polymer=40)

In [14]:
no_interactions.create_beads()
#no_interactions.print_bead_list()
len(no_interactions.bead_list)  # should be 80,000

80000

In [15]:
no_interactions.create_polymers()
#no_interactions.print_polymer_list(verbose=True)
print(len(no_interactions.polymer_list)) #should be 2000


2000


In [16]:
#no_interactions.visualize()  #too many beads?

In [17]:
r2_ave, result = r2(no_interactions)

processed polymer 0
processed polymer 500
processed polymer 1000
processed polymer 1500


In [18]:
print(r2_ave)
result 

3.400371659802691


,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.157959,0.147473,0.162123,0.171162,0.162561,0.162067,0.161183,0.154522,0.170941,0.160492,...,0.151643,0.160275,0.164059,0.162455,0.164502,0.158164,0.166980,0.166170,0.167712,0.165205
2,0.573377,0.539237,0.587038,0.622909,0.604836,0.601517,0.589440,0.558778,0.624934,0.583882,...,0.566049,0.594855,0.596864,0.602460,0.613337,0.579796,0.610138,0.605794,0.622890,0.598204
3,1.180367,1.127568,1.236802,1.296639,1.284694,1.278670,1.230059,1.142763,1.304900,1.204526,...,1.202132,1.231776,1.252014,1.271281,1.303608,1.220319,1.272031,1.268324,1.298355,1.213309
4,1.932930,1.870145,2.067931,2.125915,2.153205,2.152920,2.036066,1.851368,2.166873,1.977669,...,2.025574,2.021220,2.086195,2.127505,2.213145,2.038322,2.107563,2.116452,2.151094,1.948961
5,2.795613,2.754665,3.036090,3.037963,3.182354,3.192074,2.961980,2.648915,3.158195,2.858728,...,3.027642,2.947989,3.097328,3.140385,3.331576,3.002021,3.080127,3.099901,3.136043,2.767160
6,3.747430,3.762927,4.067861,3.982647,4.356874,4.373395,3.956739,3.500558,4.232817,3.819345,...,4.203597,4.008207,4.247694,4.278919,4.647956,4.075729,4.138215,4.172037,4.214668,3.632603
7,4.737164,4.874618,5.168995,4.875715,5.682241,5.681339,4.971550,4.374073,5.366996,4.843066,...,5.520334,5.151825,5.530251,5.497040,6.154708,5.260663,5.248536,5.304903,5.315878,4.513862
8,5.749693,6.064318,6.332719,5.669219,7.128789,7.025517,5.966913,5.254509,6.554114,5.897886,...,6.955241,6.344597,6.941923,6.770273,7.822760,6.538306,6.360340,6.506445,6.360172,5.367964
9,6.792205,7.344906,7.551715,6.308892,8.653894,8.366124,6.929147,6.109804,7.801467,6.938785,...,8.452390,7.545457,8.443768,8.057853,9.629124,7.892571,7.452251,7.751292,7.360349,6.189421


In [34]:
#r2_ave = N*b^2
kuhn_l = np.sqrt((r2_ave/no_interactions.beads_per_polymer))
print(kuhn_l)

0.29156352908940325


In [37]:
wlcave.r2_ave(kuhn_l, 3)

0.07063866230381816

Not = to 3.4  :|